In [3]:
import re
import os
import sys
import subprocess
import importlib
import requests
import urllib3

# --- 0. 환경 설정 및 라이브러리 로드 ---
def install_and_import(package):
    try:
        importlib.import_module(package)
    except ImportError:
        print(f"필수 라이브러리({package}) 설치 중...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "--user"])
        importlib.invalidate_caches()

# 필수 라이브러리 설치 (docx 추가)
install_and_import("deep_translator")
install_and_import("docx") # python-docx 설치

from deep_translator import GoogleTranslator
from docx import Document # Word 파일 처리를 위해 추가

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def force_no_ssl_verify():
    old_request = requests.Session.request
    def new_request(self, method, url, *args, **kwargs):
        kwargs['verify'] = False
        return old_request(self, method, url, *args, **kwargs)
    requests.Session.request = new_request

force_no_ssl_verify()

# --- Word 파일 읽기/쓰기 함수 ---
def read_docx(path):
    doc = Document(path)
    # 모든 단락(paragraph)의 텍스트를 줄바꿈으로 합쳐서 반환
    return "\n".join([para.text for para in doc.paragraphs])

def save_docx(text, path):
    doc = Document()
    # 텍스트를 줄바꿈 단위로 나누어 단락별로 저장
    for line in text.split('\n'):
        doc.add_paragraph(line)
    doc.save(path)

# --- I. 숫자 시퀀스 괄호 처리 로직 ---
def enclose_numbers_in_parentheses(text):
    pattern = r"(\b(?!claims?\b)[a-zA-Z]+\b)(\s\d+[a-z0-9\-\']*(?:[\s,]+(?:and|or|to|-)\s+\d+[a-z0-9\-\']*(?!\w)|[\s,]+\d+[a-z0-9\-\']*(?!\w))*)"
    
    def replacement_logic(match):
        prefix = match.group(1)
        number_sequence = match.group(2).strip() 
        return f"{prefix}({number_sequence})"

    return re.sub(pattern, replacement_logic, text, flags=re.IGNORECASE)

# --- II. 영한 번역 로직 ---
def chunk_text(text, max_size=5000):
    chunks = []
    start = 0
    while start < len(text):
        lookahead_end = min(start + max_size, len(text))
        segment = text[start:lookahead_end]
        if lookahead_end == len(text):
            if segment.strip(): chunks.append(segment.strip())
            break
        best_split_index = -1
        for char in ['.', '?', '!']:
            last_punc_index = segment.rfind(char)
            if last_punc_index > best_split_index:
                best_split_index = last_punc_index
        if best_split_index != -1:
            best_end = start + best_split_index + 1
            chunks.append(text[start:best_end].strip())
            start = best_end
        else:
            chunks.append(segment.strip())
            start = lookahead_end
    return chunks

def translate_text(text):
    translator = GoogleTranslator(source='en', target='ko')
    chunks = chunk_text(text)
    translated_chunks = []
    for i, chunk in enumerate(chunks):
        print(f"   > 번역 진행 중... ({i+1}/{len(chunks)})")
        try:
            translated_chunks.append(translator.translate(chunk))
        except Exception as e:
            translated_chunks.append(f"[번역 오류: {e}]")
    return "\n\n".join(translated_chunks)

# --- III. 비격식체 변환 로직 ---
def transform_korean_sentence(text):
    conversion_rules = {
        "입니다": "이다", "합니다": "한다", "있습니다": "있다",
        "됩니다": "된다", "냅니다": "낸다", "시킵니다": "시킨다",
        "필요한다": "필요하다", "유리한다": "유리하다",
        "했습니다": "했다" , "줍니다": "준다",
        "없습니다": "없다", "않습니다": "않는다",
        "무화과.": "도", "무화과": "도", "그림": "도",
        "매개변수": "파라미터",  "종방향": "길이방향",
        "방사상으로": "반경방향으로", "; 그리고": "; 및",
        "집니다": "진다", "바람직한다": "바람직하다"
    }
    for formal, casual in conversion_rules.items():
        text = text.replace(formal, casual)
    return text

# --- 메인 실행 프로세스 ---
def main():
    print("="*60)
    print(" 특허/기술 문서 Word 번역 프로세서 (Docx 지원)")
    print("="*60)

    # 1. 경로 설정
    input_path = input("✅ 원본 영문 Word 파일(.docx) 경로를 입력하세요: ").strip()
    if not os.path.exists(input_path):
        print("파일이 존재하지 않습니다.")
        return

    # 파일 확장자 확인
    if not input_path.lower().endswith('.docx'):
        print("이 코드는 .docx 형식만 지원합니다. (.doc 파일은 .docx로 변환 후 사용하세요)")
        return

    base_name = os.path.splitext(input_path)[0]
    final_path = input(f"✅ 최종 결과 저장 경로를 입력하세요 (엔터 시 '{base_name}_final_ko.docx'로 저장): ").strip()
    if not final_path:
        final_path = f"{base_name}_final_ko.docx"

    # --- STEP 1: 파일 읽기 및 괄호 처리 ---
    print("\n[STEP 1] Word 파일 읽기 및 숫자 시퀀스 괄호 처리 중...")
    en_text = read_docx(input_path)
    bracketed_text = enclose_numbers_in_parentheses(en_text)

    # --- STEP 2: 번역 ---
    print("\n[STEP 2] 영한 번역 진행 중 (Google)...")
    translated_text = translate_text(bracketed_text)

    # --- STEP 3: 문체 변환 및 저장 ---
    print("\n[STEP 3] 비격식체 종결형 변환 및 Word 저장 중...")
    final_text = transform_korean_sentence(translated_text)
    
    save_docx(final_text, final_path)
    
    print("\n" + "="*60)
    print(f"모든 작업이 완료되었습니다!")
    print(f"최종 저장 위치: {final_path}")
    print("="*60)

if __name__ == "__main__":
    main()


 특허/기술 문서 Word 번역 프로세서 (Docx 지원)


✅ 원본 영문 Word 파일(.docx) 경로를 입력하세요:  D:\업무\영문명세서\2026 영문명세서\IP-2026-0028-US-000.docx
✅ 최종 결과 저장 경로를 입력하세요 (엔터 시 'D:\업무\영문명세서\2026 영문명세서\IP-2026-0028-US-000_final_ko.docx'로 저장):  



[STEP 1] Word 파일 읽기 및 숫자 시퀀스 괄호 처리 중...

[STEP 2] 영한 번역 진행 중 (Google)...
   > 번역 진행 중... (1/13)
   > 번역 진행 중... (2/13)
   > 번역 진행 중... (3/13)
   > 번역 진행 중... (4/13)
   > 번역 진행 중... (5/13)
   > 번역 진행 중... (6/13)
   > 번역 진행 중... (7/13)
   > 번역 진행 중... (8/13)
   > 번역 진행 중... (9/13)
   > 번역 진행 중... (10/13)
   > 번역 진행 중... (11/13)
   > 번역 진행 중... (12/13)
   > 번역 진행 중... (13/13)

[STEP 3] 비격식체 종결형 변환 및 Word 저장 중...

모든 작업이 완료되었습니다!
최종 저장 위치: D:\업무\영문명세서\2026 영문명세서\IP-2026-0028-US-000_final_ko.docx
